In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/19 10:05
# @Author  : Wang Yujia
# @File    : data_extract_for_targets.ipynb
# @Description : 为了target datas提取信息，而且提取data_key

# 1. Preparations
## 1.1 全局设置
1. input的data包括那些duration过长的结果
2. output一共`data_key.shape[0]`+1个文件

In [3]:
# Small dataset
small_data_np_path = r'../data/small_auctions_np.csv'
settings_small_NN_path = r"../data/small_settings_NN.csv"

# Large data
large_data_np_path = r'E:\DATA\large_dta\large_auctions_np.csv'
settings_large_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'

# output path
target_output_head= "../data/targets_all/target_data_NP_"
target_output_tail= ".csv"
data_key_path = "../data/target_datakey_all.csv"

unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_endprice']

import pandas as pd
import numpy as np
from visdom import Visdom
import csv
from tqdm import tqdm
from IPython.display import clear_output

## 1.2 data读取

In [4]:
data_small = pd.read_csv(small_data_np_path,encoding="utf-8")
data_large = pd.read_csv(large_data_np_path,encoding="utf-8")

data_key_small = pd.read_csv(settings_small_NN_path,encoding="utf-8")
data_key_large = pd.read_csv(settings_large_NN_path,encoding="utf-8")

len_small = data_key_small.shape[0]
len_large = data_key_large.shape[0]
print(f"小数据集有 *{len_small}* 场auction，大数据集有 *{len_large}* 场auction，一共*{len_small+len_large}*")

小数据集有 *1196* 场auction，大数据集有 *80* 场auction，一共*1276*


## 1.3 functions about 'key'

In [9]:
def select_data_fromkey(data,data_key,i):
    return data[(data['desc'] == data_key.iloc[i,0]) &
                (data['bidincrement'] == data_key.iloc[i,1]) &
                (data['bidfee'] == data_key.iloc[i,2]) &
                (data['retail'] == data_key.iloc[i,3]) &
                (data['flg_endprice'] == data_key.iloc[i,4])].copy()

# 2. 整理与输出

1. 最后的data只有N和P，
2. 注意一定要按照`data_key`逐行保存csv，这样index可以对应


In [10]:
plot_flag = 0
if(plot_flag):
    env_str = 'N_targets'
    viz = Visdom(env=env_str)

# Col we want to save in target file
col_names = unique_setting_NN+['N','cnt_N','P']
col_NP = ['N','P']

- 先做small的

In [15]:
for i in tqdm(range(len_small)):
# for i in tqdm(range(11,12)):

    # Get data according to the data_key
    data_i = select_data_fromkey(data_small,data_key_small,i)
    # Sort data according to 'N'
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Only keep col we need
    data_i_df = data_i[col_NP]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

print("Done")

100%|██████████| 1/1 [00:00<00:00, 125.02it/s]

sum of cnt_N:3539, len of file: 1381
    auction_id  product_id                         desc  retail  price  \
0       132599    10011393  DS | Nintendo DS Lite Black  129.99   1.05   
1       129477    10011406  DS | Nintendo DS Lite Black  129.99   1.50   
2       106657    10007296  DS | Nintendo DS Lite Black  129.99   1.80   
3       142964    10011393  DS | Nintendo DS Lite Black  129.99   2.10   
4       126670    10011316  DS | Nintendo DS Lite Black  129.99   2.10   
5       134657    10011393  DS | Nintendo DS Lite Black  129.99   2.25   
6       142789    10011499  DS | Nintendo DS Lite Black  129.99   2.40   
7       134817    10010658  DS | Nintendo DS Lite Black  129.99   2.55   
8       149583    10011393  DS | Nintendo DS Lite Black  129.99   3.15   
9       143338    10011499  DS | Nintendo DS Lite Black  129.99   3.45   
10      153649    10011393  DS | Nintendo DS Lite Black  129.99   3.45   
11      143306    10011499  DS | Nintendo DS Lite Black  129.99   3.45   
1

- 大数据集

In [11]:
for i in tqdm(range(len_large)):

    # Get data according to the data_key
    data_i = select_data_fromkey(data_large,data_key_large,i)
    # Sort data according to 'N'
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Only keep col we need
    data_i_df = data_i[col_NP]
    # Get the output path
    target_output_path = target_output_head+str(i+len_small).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

print("Done")

100%|██████████| 80/80 [00:00<00:00, 586.60it/s]

Done


- concat and save key

In [6]:
data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)
data_key.to_csv(data_key_path,encoding="utf-8",header=True,index=False)
data_key.shape

# 3. 整理粒度(TARGET = 5)
1. 添加了“整理粒度”的代码，注意必须有'cnt'这个属性才可以进行粒度的整理
2. 对于DA之后的target data，重新整理粒度的代码在DA那个文件里

In [ ]:
TARGET = 5